<a href="https://colab.research.google.com/github/rz74/RF-Stage-Link-Calculator/blob/main/RF_Link_Cal_OOP_Under_Construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/rf cal/data

Mounted at /content/drive
/content/drive/MyDrive/rf cal/data


In [ ]:
pip install scikit-rf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 9.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import skrf as rf
import re
import math
from scipy.optimize import curve_fit

In [ ]:
def clean_interpolate_data(input_file, Power_Type):
    # Use Global Frequency Settings
    freq_step = step_size
    # Read the original power vs. frequency data from the CSV file without column names
    original_data = pd.read_csv(input_file, header=None, names=['freq', Power_Type])
    # output_file = input_file - '.csv'
    # output_file = output_file +'_cleaned.csv'
    output_file = 'cleaned_' + input_file
    # Ensure the data is sorted by frequency
    original_data = original_data.sort_values(by='freq')

    # Create a new frequency range with the desired step size
    new_freq_range = np.arange(freq_l, freq_h + freq_step, freq_step)

    # Interpolate the power values for the new frequency range
    interpolated_power = np.interp(new_freq_range, original_data['freq'], original_data[Power_Type])

    # Create a new DataFrame with the interpolated data
    interpolated_data = pd.DataFrame({'freq': new_freq_range, Power_Type: interpolated_power})

    # Save the cleaned and interpolated data to a new CSV file
    interpolated_data.to_csv(output_file, index=False)
    # return the cleaned df
    return interpolated_data


# user_freq_l = 6
# user_freq_h = 18
# user_freq_step = 0.5

# clean_interpolate_data(input_file, output_file, user_freq_l, user_freq_h, user_freq_step)


In [ ]:
####----####----####----####----####----####----####----USER_INPUT----####----####----####----####----####----####----####

# Define the freq range of this system in GHz
# And number of freq points
freq_l = 6
freq_h = 18
step_size = 0.5
####----####----####----####----####----####----####----END__INPUT----####----####----####----####----####----####----####


# DO NOT CHANGE
freq_num = int((freq_h - freq_l) / step_size) + 1
f = np.linspace(freq_l,freq_h,freq_num)
f_df = pd.DataFrame(f, columns=['Frequency'])

In [ ]:
input_file = 'sample.csv'

user_freq_l = 6
user_freq_h = 18
user_freq_step = 0.5

clean_interpolate_data(input_file, 'Psat')

In [ ]:
input_file = 'sample.csv'

user_freq_l = 6
user_freq_h = 18
user_freq_step = 0.5

clean_interpolate_data(input_file, 'Psat')

,freq,Psat
0,6.0,44.155833
1,6.5,44.516185
2,7.0,44.462238
3,7.5,44.321831
4,8.0,44.009358
5,8.5,43.437794
6,9.0,43.830358
7,9.5,44.208926
8,10.0,44.517838
9,10.5,45.846905


In [ ]:
# def linear_fit(x, a, b):
#     return a * x + b

# def find_p1db(input_file, output_file, freq_column, pin_column, pout_column):
#     # Read the input CSV file
#     data = pd.read_csv(input_file)

#     # Get unique frequencies in the data
#     unique_freqs = data[freq_column].unique()

#     # Initialize a DataFrame to store results
#     results = pd.DataFrame(columns=['Frequency', 'P1dB'])

#     for freq in unique_freqs:
#         # Filter data for the current frequency
#         freq_data = data[data[freq_column] == freq]

#         # Sort data by input power
#         freq_data = freq_data.sort_values(by=pin_column)

#         # Fit a linear line to the starting data points
#         popt, _ = curve_fit(linear_fit, freq_data[pin_column], freq_data[pout_column])

#         # Calculate P1dB point (output power where input power is 1 dB below the maximum)
#         max_input_power = freq_data[pin_column].max()
#         p1db_input_power = max_input_power - 1.0
#         p1db_output_power = linear_fit(p1db_input_power, *popt)

#         # Append results to the DataFrame
#         results = results.append({'Frequency': freq, 'P1dB': p1db_output_power}, ignore_index=True)

#         # Plot the linear fit for illustration
#         plt.plot(freq_data[pin_column], linear_fit(freq_data[pin_column], *popt), label=f'Frequency: {freq}')

#     # Save the plot
#     plt.legend()
#     plt.xlabel('Input Power (dBm)')
#     plt.ylabel('Output Power (dBm)')
#     plt.savefig('linear_fit_plot.png')
#     plt.show()


#     results.to_csv(output_file, index=False)




In [ ]:
# DO NOT CHANGE
# input: the ntwk object and freq range and number of points
# output: a panda data frame with gain in db form

def ntwk_to_db(input_ntwk):


  # data cleaning
  # convert a ntwk to it's gain in db
  # bt/w lf and hf (in GHz)

  # define freq range and interpolate missing points

  f = np.linspace(freq_l,freq_h,freq_num)
  freq = rf.Frequency.from_f(f, unit='GHz')
  input_ntwk.interpolate_self(freq)
  output_ntwk = input_ntwk['6-18ghz']

  # pull s21 from ntwk
  output_s21 = output_ntwk.s21
  output_db = output_s21.s_db

  # convert to panda df
  input_string = str(input_ntwk)

  # pattern to match the sub string between single quotes
  pattern = r"'([^']+)'"

  # find the first match in the string
  match = re.search(pattern, input_string)

  # Check if a match is found
  if match:
      # Extract the matched substring
      result = match.group(1)
      result = result+'_Gain'
      #print(result)
  else:
      pass
      #print("No match found")
  #
  gain = output_db[:,0,0]
  gain.flatten()
  gain_df = pd.DataFrame(gain, columns=[result])

  gain_df.insert(0,'Frequency',f_df['Frequency'])
  return gain_df

In [ ]:
class RFStage:
    def __init__(self, s_parameters_file, power_vs_freq_file):
        # Reading S parameters from an S2P file
        self.s_parameters = rf.Network(s_parameters_file)

        # Calculating gain based on S parameters
        self.calculate_gain()

        # Reading power vs. frequency data from a CSV file
        self.power_vs_freq = pd.read_csv(power_vs_freq_file)

        # Calculating PSAT and P1dB based on power vs. frequency
        self.calculate_psat()
        # self.P1dB = self.calculate_p1db()
        # User-defined frequency values
        self.freq_h = freq_h
        self.freq_l = freq_l
        self.freq_step = step_size

    def calculate_gain(self):

        self.Gain = ntwk_to_db(self.s_parameters)

    def calculate_psat(self):

        self.Psat = clean_interpolate_data(power_vs_freq_file, "Psat")

    def print_summary(self):
        # A function to print a summary of the RF stage
        print("Summary:")
        # print("S Parameters:")
        # print(self.s_parameters)
        print("\nGain:")
        print(self.Gain)
        # print("\nPower vs. Frequency:")
        # print(self.power_vs_freq)
        print("\nPSAT:", self.Psat)
        # print("\nP1dB:", self.p1db)
        print("\nConstants(Unit: GHz):")
        print("Frequency High:", self.freq_h)
        print("Frequency Low:", self.freq_l)
        print("Frequency Step:", self.freq_step)




In [ ]:
s_parameters_file = 'sample.s2p'
power_vs_freq_file = 'sample.csv'
user_freq_h = 18
user_freq_l = 6
user_freq_step = 0.5

TGA2963 = RFStage(s_parameters_file, power_vs_freq_file)

# Call the print_summary function
TGA2963.print_summary()

Summary:

Gain:
    Frequency  TGA2963_Gain
0         6.0        28.930
1         6.5        28.518
2         7.0        27.832
3         7.5        27.916
4         8.0        28.415
5         8.5        29.051
6         9.0        30.014
7         9.5        31.522
8        10.0        33.455
9        10.5        35.360
10       11.0        36.103
11       11.5        35.593
12       12.0        34.632
13       12.5        33.599
14       13.0        32.692
15       13.5        31.956
16       14.0        31.686
17       14.5        32.088
18       15.0        33.041
19       15.5        34.691
20       16.0        35.836
21       16.5        31.813
22       17.0        28.333
23       17.5        27.114
24       18.0        26.759

PSAT:     freq       Psat
0    6.0  44.155833
1    6.5  44.516185
2    7.0  44.462238
3    7.5  44.321831
4    8.0  44.009358
5    8.5  43.437794
6    9.0  43.830358
7    9.5  44.208926
8   10.0  44.517838
9   10.5  45.846905
10  11.0  46.270389
11  11.5 

In [ ]:
TGA2963.Psat

In [ ]:
TGA2963.Gain